$\DeclareMathOperator*{\E}{\mathbb{E}}$
$\DeclareMathOperator*{\Q}{\mathbb{Q}}$


$\DeclareMathOperator*{\E}{\mathbb{E}} % expectation
\DeclareMathOperator*{\Q}{\mathbb{Q}} % Q probability measure
\DeclareMathOperator*{\F}{{\mathcal{F}}}
\DeclareMathOperator{\d}{\mathrm{d}\!}  % differential
$

# Heston-stochastic-volatility-model

In the notebooks I'll discuss various implementations of the Heston stochastic volatility model of pricing European call options. 

The Heston model goes beyond the Black-Scholes options pricing model in the sense that the constant volatility of the Black-Scholes model is replaced by a stochastic process for the volatility. This simple trick makes it possible to account for volatility smiles often seen in practice.

## Mathematical description of the model

The Heston stochastic volatility model is described by two coupled stochastic differential equations: one for the stock process, and another for the volatility process both driven by their respective Brownian motions:

\begin{align} 
\d S_t & = \sqrt{v_t} S_t \d B_t, \qquad S_0 > 0 \\
\d v_t & = \kappa (\theta-r_t) \d t + \sigma\sqrt{v_t}(\rho \d B_t + \sqrt{1-\rho^2} \d W_t), \qquad v_0 > 0.
\end{align}

In these two coupled stochastic differential equations:
* $S_t$ is stock price process driven by the $B_t$ Brownian motion, 
* $v_t$ is the volatility process driven by $W_t$ Brownian motion ~~mixed with $B_t$ so that the volatility is driven by a Brownian motion that has $\rho$ correlation with $B_t$. $W_t$ is independent of $B_t$~~.

The stock price process is not a geometric Brownian motion since the volatility is a non-adapted stochastic process (instead of being constant or deterministic as in the Black-Scholes case).

The volatility process is a CIR process, a mean reversion process which has
* mean-reversion level $\theta$
* the volatility mean revert to this level at the mean-reversion speed $\kappa$
* $\sigma$ is volatility of the volatility process (vol of vol).

$B$ and $W$ are two independent $\Q$-Brownian motions generating the filtration $\F_t$ in the ($\Omega$, $\F_t$, $\Q$) probability space, where $\Q$ is the pricing measure.

Time $t$ is in the $[0, T]$ interval where $T$ is the expiry time of the option measured in years.

The price of a call option, whose strike price is $K$, $c(t, s, v)$ is the conditional expectation of the final pay-off (random variable) $(S_T - K)^+$  given the $\F_t$ filtration at time $t$, i.e., given all information up to $t$ in the $\Q$ pricing measure:

$$ c(t, S_t, v_t) = {\E}^{\Q}[(S_T - K)^+|{\F}_t] $$

In the following Monte Carlo simulation, I'll generate the path of the stock price $S_t$ and its volatility, $v_t$ from $t=0$ to $T$, so $\F_t = \F_0$, that is, no information is known after $t=0$, hence the conditional expectation above becomes a simple expectation with no condition, since ${\E}^{\Q}[(S_T - K)^+|{\F}_{t=0}] = {\E}^{\Q}[(S_T - K)^+]$




## Monte Carlo simulation

I'll show how to integrate the coupled SDE's using the simplest integration method, the Euler method, generating MC paths and then making averages.

In the first pass, I'll show the MC path generation using for loops to make things more accessible, and later give a vectorized solution which can utilize the capabilities of CPU's or GPU's vector units making the algorithm 100x (?) faster.

So, let's discretize the the SDEs: 
* $\Delta$ is the time step 
* $Z'$   standard normal distributions for the $S_t$ process and 
* $Z''$ standard normal distribution independent of $Z'$, which helps make
* $Z$ driving $v_t$ where $Z = \rho Z' + \sqrt{1-\rho^2}Z''$ correlated standard normal -- its correlation with $Z'$ is $\rho$

Since $v_t$ cannot be negative, I'll replace a negative $v$ with $v=0$, which is called the absorbing way. (It is left to the reader as an exercise to try the reflection way, when a negative $v$ is replaced by $v=-v$. Try and see what you get using this way.)

### Explicit Euler scheme

\begin{align}
S_{i+1} & = S_i + \sqrt{v_i} S_i \sqrt{\Delta} Z'   = S_i \exp[\sqrt{v_i}\sqrt{\Delta} Z' - v_i \Delta /2]\\
v_{i+1} & = v_i + \kappa (\theta - v_i) \Delta + \sigma\sqrt{v_i}\sqrt{\Delta}Z
\end{align}


In [39]:
import random
from math import sqrt, exp

In [40]:
random.seed(42)  # Seed random number generator

In [41]:
N = 1000    # Number of time steps
n = 10000  # Number of MC paths (10,000 20sec on MBP18 1 thread)

S0 = 100.   # Initial price of share of stock
K = 100.    # Strike price of option
v0 = 0.01   # Initial volatility
th = 0.01   # Long time average of volatility
k = 2.      # kappa mean reversion speed
sig = 0.1   # vol of vol

rho = -.5   # correlation coefficient of Z' and Z
T = 1.      # expiration time from today

dt = T/N    # Time step

In [42]:
# Integrate equations:
#   Euler method
#   for loop based (not vectorized)
Stv = [0.] * n  # Reserve St vector of size n
for i in range(n):  # i-th MC path
  vt = v0
  St = S0
  for t in range(1,N):  # Generate a MC path

    Zs = random.gauss(0.,1.)  # Generate rnd numbers (Mersenne Twister algo) for S_t process
    Zv = rho * random.gauss(0.,1.) + sqrt(1 - rho**2.) * random.gauss(0., 1.)  # Rnd number for v_t process

    # Euler integration
    vt = max(vt, 0.)
    St = St * exp( sqrt(vt * dt) * Zs - vt * dt / 2.)          # Stock price process
    vt = vt + k * (th - vt) * dt + sig * sqrt(vt * dt) * Zv    # Volatility process
  Stv[i] = St  # Store S_T's

In [22]:
sum_m = lambda x: max(x - K, 0.)
sum_s = lambda x: max(x - K, 0.) ** 2.

mean_price = sum(map(sum_m, Stv)) / n
std_error_price  = sqrt(sum(map(sum_s, Stv)) / (n * (n - 1.)))

In [23]:
mean_price, std_error_price

(4.065066582770173, 0.07505415757561733)

#### Vectorized Monte Carlo / Numpy

In [44]:
# Quick numpy/python refresher from Karpathy at Stanford 
#   http://cs231n.github.io/python-numpy-tutorial/
# Quick comparison of numpy and Matlab, which might be useful 
# if you are coming from the Matlab world 
#   https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html
import numpy as np

In [45]:
np.random.seed(42)  # Seed random number generator

In [46]:
N = 1000    # Number of time steps
n = 1000000  # Number of MC paths (1,000,000  2min on Colab; 80sec on MBP18 6 threads)

S0 = 100.   # Initial price of share of stock
K = 100.    # Strike price of option
v0 = 0.01   # Initial volatility
th = 0.01   # Long time average of volatility
k = 2.      # kappa mean reversion speed
sig = 0.1   # vol of vol

rho = -.5   # correlation coefficient of Z' and Z
T = 1.      # expiration time from today

dt = T/N    # Time step

In [47]:
# Integrate equations:
#   Euler method
#   MC vectorized
vt = np.ones(n) * v0
St = np.ones(n) * S0
for t in range(1,N):  # Generate MC paths

  Zs = np.random.normal(size=n)  # Generate rnd numbers (??? rnd gen) for S_t process
  Zv = rho * Zs + sqrt(1 - rho**2.) * np.random.normal(size=n)  # Rnd numbers for v_t process

  # Euler integration
  vt = np.maximum(vt, 0.)
  St = St * np.exp( np.sqrt(vt * dt) * Zs - vt * dt / 2.)     # Stock price process
  vt = vt + k * (th - vt) * dt + sig * np.sqrt(vt * dt) * Zv  # Volatility process

mean_price = np.mean(np.maximum(St - K, 0.))
std_error_price  = np.std(np.maximum(St - K, 0.)) / sqrt(n)

In [48]:
mean_price, std_error_price

(3.9139916791665783, 0.005651208214228738)

#### Vectorized Monte Carlo / TensorFlow

In [ ]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
tf.version.VERSION

In [ ]:
tf.executing_eagerly()

In [ ]:
tf.random.set_random_seed(42)

In [ ]:
N = 1000    # Number of time steps
n = 1000000  # Number of MC paths (1,000,000  30sec on Colab)

S0 = 100.   # Initial price of share of stock
K = 100.    # Strike price of option
v0 = 0.01   # Initial volatility
th = 0.01   # Long time average of volatility
k = 2.      # kappa mean reversion speed
sig = 0.1   # vol of vol

rho = -.5   # correlation coefficient of Z' and Z
T = 1.      # expiration time from today

dt = T/N    # Time step

In [ ]:
# Integrate equations:
#   Euler method
#   MC vectorized
vt = tf.ones(n) * v0
St = tf.ones(n) * S0
for t in range(1,N):  # Generate MC paths

  Zs = tf.random.normal([n])  # Generate rnd numbers (??? rnd gen) for S_t process
  Zv = rho * Zs + tf.sqrt(1 - rho**2.) * tf.random.normal([n])  # Rnd numbers for v_t process

  # Euler integration
  vt = tf.maximum(vt, 0.)
  St = St * tf.exp( tf.sqrt(vt * dt) * Zs - vt * dt / 2.)     # Stock price process
  vt = vt + k * (th - vt) * dt + sig * tf.sqrt(vt * dt) * Zv  # Volatility process

mean_price = tf.reduce_sum(tf.maximum(St - K, 0.)) / n
std_error_price  = np.std(tf.maximum(St - K, 0.)) / tf.sqrt(float(n))

In [ ]:
mean_price, std_error_price